In [18]:
%pip -q install google-genai

In [19]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [20]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [21]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição será realizada é:

*   **Acompanhar as redes sociais da Alura:** Fique de olho nos perfis da Alura no Instagram, LinkedIn, Twitter e Facebook. Eles costumam anunciar novos cursos e imersões por lá.
*   **Assinar a newsletter da Alura:** Ao assinar a newsletter, você receberá e-mails com as novidades e lançamentos da Alura, incluindo informações sobre a Imersão IA com Google Gemini.
*   **Verificar a página de cursos da Alura:** Acesse o site da Alura e verifique a seção de cursos de Inteligência Artificial. A Imersão IA com Google Gemini pode aparecer lá quando estiver com inscrições abertas.
*   **Comunidade da Alura:** Interaja com a comunidade da Alura no Discord ou fórum. Outros alunos podem ter informações sobre a próxima edição da Imersão.

Seguindo essas dicas, você estará bem informado e poderá se inscrever assim que a próxima Imersão IA com Google Gemini for anunciada.

In [22]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    # Inserir a tool de busca do Google ###############################################
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A Imersão IA Google Gemini mais recente ocorreu de 12 a 16 de maio de 2025. As inscrições para esta imersão estiveram abertas até 11 de maio de 2025.

A Imersão IA é um curso online e gratuito que ensina a usar o Google Gemini e não exige conhecimento prévio. Ao completar o curso, os participantes ganham um certificado da Alura e do Google.

In [23]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima imersão IA com google gemini alura']
Páginas utilizadas na resposta: starten.tech, youtube.com, alura.com.br



In [24]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk


In [25]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [26]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [27]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [28]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do Google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)

    return lancamentos

In [29]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Vocé é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes do buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes , e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [30]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [31]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [33]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################

if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}.")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("------------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("------------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("------------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("------------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: Agentes de IA
Maravilha! Vamos então criar o post sobre novidades em Agentes de IA.

--- Resultado do Agente 1 (Buscador) ---



> Aqui estão algumas das notícias e lançamentos mais recentes sobre agentes de IA:
> 
> 
> Com base nas notícias mais recentes, aqui estão 5 lançamentos e atualizações relevantes sobre agentes de IA:
> 
> 1.  **IBM watsonx Orchestrate:** A IBM anunciou atualizações para o watsonx Orchestrate, oferecendo soluções para integrar, personalizar e implantar agentes de IA. Ele também inclui o Agent Catalog, que simplifica a integração com aplicações empresariais como Adobe, AWS, Microsoft, Oracle, Salesforce, SAP, ServiceNow e Workday.
> 2.  **NVIDIA AI Data Platform e Microsserviços NIM:** A NVIDIA lançou uma família de modelos abertos de IA com capacidade de raciocínio, juntamente com a NVIDIA AI Data Platform e os microsserviços NIM. Esses lançamentos visam capacitar desenvolvedores e empresas a criar plataformas de agentes de IA.
> 3.  **Microsoft Copilot com Agentes Researcher e Analyst:** A Microsoft anunciou novos agentes de IA para o Microsoft 365 Copilot, chamados Researcher e Analyst. O Researcher é projetado para agilizar a criação de relatórios, enquanto o Analyst é usado para refinar dados.
> 4.  **Agentes de IA na B3:** A B3 (Brasil, Bolsa, Balcão) adotou o uso de agentes de IA para revolucionar suas operações em 2025. Ao contrário dos assistentes de IA que aguardam comandos, os agentes de IA podem planejar e executar tarefas de forma independente, integrando diferentes sistemas com mínima intervenção humana.
> 5.  **Integração e Análise com Apiiro e ServiceNow:** A Apiiro firmou uma parceria estratégica com a ServiceNow para integrar sua plataforma de análise de código com o CMDB da ServiceNow, visando oferecer um inventário preciso dos ativos de software e otimizar a eficiência operacional impulsionada por IA.


------------------------------------------------------------------

--- Resultado do Agente 2 (Planejador) ---



> Okay, com base nos lançamentos mais recentes sobre agentes de IA, vou pesquisar mais a fundo para identificar o tema mais relevante e criar um plano de conteúdo para um post.
> 
> 
> Com base nas notícias e lançamentos recentes, o tema mais relevante para um post sobre Agentes de IA é:
> 
> **Adoção de Agentes de IA em Empresas Brasileiras e a Transformação do Mercado de Trabalho.**
> 
> **Pontos Relevantes:**
> 
> *   **B3 (Brasil, Bolsa, Balcão) e a Adoção de Agentes de IA:** A B3 está na vanguarda da adoção de agentes de IA para otimizar suas operações e melhorar a experiência do cliente. Ao contrário dos assistentes de IA que aguardam comandos, os agentes de IA podem planejar e executar tarefas de forma independente, integrando diferentes sistemas com mínima intervenção humana. Eles são usados para aumentar a produtividade e entregar melhores experiências em produtos e serviços.
> *   **Exemplo Prático: Digital Coach da B3:** Um exemplo concreto é o "Digital Coach", um agente de IA que monitora as centrais de atendimento, analisa a qualidade das informações fornecidas e a eficácia na resolução de problemas, fornecendo insights para melhorias contínuas.
> *   **Impacto nos Colaboradores:** A implementação de agentes de IA permite que os colaboradores da B3 se concentrem em atividades mais analíticas e estratégicas, delegando tarefas processuais e operacionais aos agentes.
> *   **Cenário Global e Investimentos:** O investimento global em IA está se aproximando de US$ 750 bilhões, sinalizando a importância crescente dessa tecnologia.
> *   **Diferença entre Assistentes e Agentes de IA:** É crucial distinguir entre assistentes de IA (reativos a comandos) e agentes de IA (autônomos e proativos).
> *   **Tendências de Mercado:** Empresas como IBM, Microsoft e NVIDIA estão lançando plataformas e ferramentas para facilitar a criação e implementação de agentes de IA.
> 
> **Plano de Conteúdo:**
> 
> 1.  **Título:** Agentes de IA Transformam a B3 e o Futuro do Trabalho no Brasil
> 2.  **Introdução:**
>     *   O que são agentes de IA e como eles se diferenciam dos assistentes de IA.
>     *   A crescente adoção de agentes de IA em diversos setores.
>     *   A importância do investimento em IA no cenário global.
> 3.  **Case da B3:**
>     *   Como a B3 está utilizando agentes de IA para otimizar suas operações.
>     *   O papel do Digital Coach e seus benefícios.
>     *   O impacto dos agentes de IA na produtividade e no trabalho dos colaboradores.
> 4.  **Perspectivas Futuras:**
>     *   Como a adoção de agentes de IA pode impactar outras empresas brasileiras.
>     *   As tendências de mercado e os lançamentos de novas plataformas e ferramentas.
>     *   Oportunidades e desafios da implementação de agentes de IA.
> 5.  **Conclusão:**
>     *   Reforçar a importância da adoção de agentes de IA para a transformação digital das empresas.
>     *   Incentivar a discussão sobre o futuro do trabalho e o papel da IA na sociedade.
> 
> Este plano de conteúdo visa fornecer uma visão abrangente sobre a adoção de agentes de IA no Brasil, utilizando o caso da B3 como exemplo prático e relevante. Além disso, explora as tendências de mercado e os desafios da implementação dessa tecnologia, incentivando a reflexão sobre o futuro do trabalho e o papel da IA na sociedade.
> 


------------------------------------------------------------------

--- Resultado do Agente 3 (Redator) ---



> 🚀 **Revolução na B3: Agentes de IA Transformando o Mercado de Trabalho!** 🤖
> 
> Já ouviu falar em Agentes de IA? 🧐 Eles são a nova onda tecnológica que está transformando empresas como a B3 (Brasil, Bolsa, Balcão)! 📈
> 
> **Mas qual a diferença entre Agentes e Assistentes de IA?** 🤔
> 
> Enquanto os assistentes de IA esperam por comandos, os agentes de IA são **autônomos e proativos**, planejando e executando tarefas com mínima intervenção humana. 🤯
> 
> **Na B3,** um exemplo prático é o "Digital Coach", um agente de IA que monitora as centrais de atendimento, analisa a qualidade das informações e ainda oferece insights para melhorias contínuas! 🤩
> 
> **E o impacto disso nos colaboradores?** 🎯 Eles podem focar em atividades mais estratégicas e analíticas, delegando tarefas operacionais aos agentes. 🧠
> 
> Com o investimento global em IA se aproximando de **US$ 750 bilhões**, empresas como IBM, Microsoft e NVIDIA estão lançando ferramentas para facilitar a criação e implementação de agentes de IA. 💰
> 
> Prepare-se, porque essa tecnologia está transformando o futuro do trabalho! 🚀
> 
> Quer saber mais sobre como os agentes de IA podem revolucionar sua empresa? Deixe um comentário! 👇
> 
> #AgentesDeIA #Inovação #TransformaçãoDigital #Alura
> 


------------------------------------------------------------------

--- Resultado do Agente 4 (Revisor) ---



> O rascunho está ótimo e pronto para publicar!
> 


------------------------------------------------------------------
